# 머신러닝 with Python 
## 09 앙상블 학습 배깅(Bootstrap aggregating) 실습- wine data set

##### Random Forest 알고리즘을 이용하여 와인 데이터를 분류하는 모형을 만들어본다.

책 p.267~ 

---

하단부에서는 최대 레벨 수를 변경하면서 실험해 보았다.

> 배운점1: 뎁스가 낮으면 분류의 정확도가 낮아진다.

> 배운점2: 일정 뎁스 이상에서는 분류의 정확도가 더 이상 상승하지 않는다.

### 데이터 불러오기

In [1]:
from sklearn import datasets 

In [2]:
raw_wine = datasets.load_wine() # 와인 데이터 가져오기

In [3]:
# 데이터 살펴보기

raw_wine

{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
         8.350e+02],
        [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
         8.400e+02],
        [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
         5.600e+02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [4]:
raw_wine.feature_names      # 데이터 셋 내 피처 이름들

['alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines',
 'proline']

### 피처, 타깃 데이터 지정

In [5]:
X = raw_wine.data
y = raw_wine.target

### 트레이닝/테스트 데이터 분할

In [6]:
from sklearn.model_selection import train_test_split              # 분할을 위해 필요한 함수
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state = 0) # 분리. randomstate는 고정

### 데이터 표준화

In [7]:
from sklearn.preprocessing import StandardScaler # 데이터 표준화를 위한 함수
std_scale = StandardScaler()                     # 표준화 스케일러 지정
std_scale.fit(X_tn)                              # 트레이닝 피처를 기준으로 표준화를 적합 시도

X_tn_std = std_scale.transform(X_tn) 
X_te_std = std_scale.transform(X_te)             # 트레인, 테스트 데이터 각각 적합시킨 표준화에 맞게 변형

### 데이터 학습

In [8]:
from sklearn.ensemble import RandomForestClassifier     # 랜덤 포레스트 함수 불러오기
clf_rf = RandomForestClassifier(max_depth = 2,          # 분류 문제이므로 Classifier를 가져오지만, 회귀문제라면 Regressor를 데려온다
                                random_state = 0)       # max_depth: 트리의 최대 깊이(레벨). 2로 설정한다.

clf_rf.fit(X_tn_std, y_tn)

RandomForestClassifier(max_depth=2, random_state=0)

### 데이터 예측

In [9]:
pred_rf = clf_rf.predict(X_te_std)                      # std된 테스트 피처 데이터를 넣고 실행하여 결과 확인 
print(pred_rf)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 2 0 0 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


### 정확도 평가

In [10]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_rf)        # 실제값과 예측값을 넣음
print(accuracy)

0.9555555555555556


### confusion matrix 확인

In [11]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_rf)
print(conf_matrix)

[[16  0  0]
 [ 1 19  1]
 [ 0  0  8]]


### 분류 리포트 확인

In [12]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_rf)
print(class_report)

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        16
           1       1.00      0.90      0.95        21
           2       0.89      1.00      0.94         8

    accuracy                           0.96        45
   macro avg       0.94      0.97      0.95        45
weighted avg       0.96      0.96      0.96        45



### 결과

F1-score를 참조하면 모델의 성능은 0.96 으로 볼 수 있겠다.

# 실험: 뎁스 조정

## 실험1: 최대 레벨 낮추기 (2 $\rightarrow$ 1)

### 데이터 학습

In [13]:
from sklearn.ensemble import RandomForestClassifier           # 랜덤 포레스트 함수 불러오기
clf_rf_dep1 = RandomForestClassifier(max_depth = 1,           # 분류 문제이므로 Classifier를 가져오지만, 회귀문제라면 Regressor를 데려온다
                                     random_state = 0)

clf_rf_dep1.fit(X_tn_std, y_tn)

RandomForestClassifier(max_depth=1, random_state=0)

### 데이터 예측

In [15]:
pred_rf_dep1 = clf_rf_dep1.predict(X_te_std)                      # std된 테스트 피처 데이터를 넣고 실행하여 결과 확인 
print(pred_rf_dep1)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 2 0 0 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 1 0 1 1 1]


### 정확도 평가

In [16]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_rf_dep1)        # 실제값과 예측값을 넣음
print(accuracy)

0.9333333333333333


### confusion matrix 확인

In [17]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_rf_dep1)
print(conf_matrix)

[[15  1  0]
 [ 1 19  1]
 [ 0  0  8]]


### 분류 리포트 확인

In [18]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_rf_dep1)
print(class_report)

              precision    recall  f1-score   support

           0       0.94      0.94      0.94        16
           1       0.95      0.90      0.93        21
           2       0.89      1.00      0.94         8

    accuracy                           0.93        45
   macro avg       0.93      0.95      0.94        45
weighted avg       0.93      0.93      0.93        45



### 결과 

실제로 1개를 더 틀리면서 성능이 낮아졌다. 최대 레벨이 2일 때보다 약 0.02 퍼센트 포인트 정도 성능이 낮아졌다.

class 0을 1로 분류하는 오류가 생겼다.

## 실험2: 최대 레벨 올리기 (2 $\rightarrow$ 3)

### 데이터 학습

In [19]:
from sklearn.ensemble import RandomForestClassifier           # 랜덤 포레스트 함수 불러오기
clf_rf_dep3 = RandomForestClassifier(max_depth = 3,           # 분류 문제이므로 Classifier를 가져오지만, 회귀문제라면 Regressor를 데려온다
                                     random_state = 0)

clf_rf_dep3.fit(X_tn_std, y_tn)

RandomForestClassifier(max_depth=3, random_state=0)

### 데이터 예측

In [20]:
pred_rf_dep3 = clf_rf_dep3.predict(X_te_std)                      # std된 테스트 피처 데이터를 넣고 실행하여 결과 확인 
print(pred_rf_dep3)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 2 0 1 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


### 정확도 평가

In [21]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_rf_dep3)        # 실제값과 예측값을 넣음
print(accuracy)

0.9777777777777777


### confusion matrix 확인

In [22]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_rf_dep3)
print(conf_matrix)

[[16  0  0]
 [ 0 20  1]
 [ 0  0  8]]


### 분류 리포트 확인

In [23]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_rf_dep3)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      0.95      0.98        21
           2       0.89      1.00      0.94         8

    accuracy                           0.98        45
   macro avg       0.96      0.98      0.97        45
weighted avg       0.98      0.98      0.98        45



### 결과

최대 레벨이 1일 때는 0.9333, 2일 때에는 0.9555, 3일 때에는 0.9777로 성능이 조금씩 향상되었다.

그럼 한 단계 더 높인다면?

## 실험3: 최대 레벨 더 올리기 (2 $\rightarrow $ 4)

### 데이터 학습

In [24]:
from sklearn.ensemble import RandomForestClassifier           # 랜덤 포레스트 함수 불러오기
clf_rf_dep4 = RandomForestClassifier(max_depth = 4,           # 분류 문제이므로 Classifier를 가져오지만, 회귀문제라면 Regressor를 데려온다
                                     random_state = 0) 

clf_rf_dep4.fit(X_tn_std, y_tn)

RandomForestClassifier(max_depth=4, random_state=0)

### 데이터 예측

In [26]:
pred_rf_dep4 = clf_rf_dep4.predict(X_te_std)                      # std된 테스트 피처 데이터를 넣고 실행하여 결과 확인 
print(pred_rf_dep4)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 2 0 1 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


### 정확도 평가

In [27]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_rf_dep4)        # 실제값과 예측값을 넣음
print(accuracy)

0.9777777777777777


### confusion matrix 확인

In [28]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_rf_dep4)
print(conf_matrix)

[[16  0  0]
 [ 0 20  1]
 [ 0  0  8]]


### 분류 리포트 확인

In [29]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_rf_dep4)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      0.95      0.98        21
           2       0.89      1.00      0.94         8

    accuracy                           0.98        45
   macro avg       0.96      0.98      0.97        45
weighted avg       0.98      0.98      0.98        45



### 결과

정확도는 최대 레벨이 3일 때보다 더 올라가진 않았다. 한번 더 올리게 되면 1.0 에 가까운 상황이 나올까?

## 실험4: 최대 레벨 더 올려보기 (5이상 반복문)

> 최대 레벨을 5이상으로 10 정도까지 올려보는데 그 중 정확도가 가장 높은 뎁스 중 최소의 뎁스를 가진 모델을 가져와보도록 해보았다.

### 데이터 학습

In [34]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


pre_accuracy = 0
min_max_depth = 0
for i in range(5, 11):
    # 모델 빌딩
    clf_rf = RandomForestClassifier(max_depth = i, random_state = 0) 
    
    # 데이터 학습
    clf_rf.fit(X_tn_std, y_tn)
    
    # 데이터 예측
    pred_rf = clf_rf_dep.predict(X_te_std)
    
    # 정확도 평가
    accuracy = accuracy_score(y_te, pred_rf)
    
    if accuracy > pre_accuracy:
        pre_accuracy = accuracy
        min_max_depth = i
    elif accuracy <= pre_accuracy:
        pass

    print('현재 max_depth:', i)
    print('현재 accuracy:', accuracy)
    

현재 max_depth: 5
현재 accuracy: 0.9777777777777777
현재 max_depth: 6
현재 accuracy: 0.9777777777777777
현재 max_depth: 7
현재 accuracy: 0.9777777777777777
현재 max_depth: 8
현재 accuracy: 0.9777777777777777
현재 max_depth: 9
현재 accuracy: 0.9777777777777777
현재 max_depth: 10
현재 accuracy: 0.9777777777777777


In [35]:
print("최대 정확도를 위한 최소 레벨수", min_max_depth)
print("최종 정확도",pre_accuracy)

최대 정확도를 위한 최소 레벨수 5
최종 정확도 0.9777777777777777


### 데이터 예측

In [36]:
pred_rf = clf_rf_dep.predict(X_te_std)                      # std된 테스트 피처 데이터를 넣고 실행하여 결과 확인 
print(pred_rf) 

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 2 0 1 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


### 정확도 평가

In [37]:
accuracy = accuracy_score(y_te, pred_rf)        # 실제값과 예측값을 넣음
print(accuracy)

0.9777777777777777


### confusion matrix 확인

In [38]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_rf)
print(conf_matrix)

[[16  0  0]
 [ 0 20  1]
 [ 0  0  8]]


### 분류 리포트 확인

In [39]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_rf)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      0.95      0.98        21
           2       0.89      1.00      0.94         8

    accuracy                           0.98        45
   macro avg       0.96      0.98      0.97        45
weighted avg       0.98      0.98      0.98        45



일정 뎁스가 되면 더 이상 정확도가 상승하지 않는 것을 확인하였다. 

캐글같은 데이터 분석 경진대회에서 주어진 데이터 셋에 대해 성능을 최대로 올리기 위해 랜덤포레스트 방식을 선호한다는 것을 들었던 것 때문에 혹시나 1.0이 나오는 일이 있을까 싶어서 확인해 보았다.